# Running a Federated Cycle with Synergos

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

![Simulated Synergos Basic Grid](../../docs/images/synbasic_setup.png "A simple Synergos Basic setup")

This tutorial aims to give you an understanding of how to use the `Synergos` driver package to run a full federated learning cycle on a `Synergos Basic` grid.

In a `Synergos Basic` grid, this is TTP at its purest form, synchronizing with all worker nodes to facilitate federated orchestration.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on leaf texture, comprising 64 predictor features, and 1 target feature for 16 classes. The dataset is available in the same directory as this notebook. Within the dataset directory, `data1` is for Worker 1 and `data2` is for Worker 2. The task to be carried out will be multi-classification.

The dataset we have provided is a processed subset of the [original One-hundred plant species leaves dataset](https://archive.ics.uci.edu/ml/datasets/One-hundred+plant+species+leaves+data+set).

**Reference:**
- *Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.*
- *Charles Mallah, James Cope, James Orwell. Plant Leaf Classification Using Probabilistic Integration of Shape, Texture and Margin Features. Signal Processing, Pattern Recognition and Applications, in press. 2013.*

## Initiating the docker containers

Before we begin, we have to start the docker containers.

### A. Initialization via `Synergos Simulator`

In `Synergos Simulator`, a sandboxed environment has been created for you!

By running:

    `docker-compose -f docker-compose-synbasic.yml up --build`
    
the following components will be started:
- TTP (Basic)
- Worker_1
- Worker_2
- Synergos UI

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings. 

### B. Manual Initialization

Firstly, pull the required docker images with the following commands:

1. Synergos TTP (Basic): 

    `docker pull gcr.io/synergos-aisg/synergos_ttp:v0.1.0`

2. Synergos Worker: 

    `docker pull gcr.io/synergos-aisg/synergos_worker:v0.1.0`

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**TTP**

```
docker run 
    -p 5000:5000 
    -p 8020:8020
    -v <directory leaf_textures/ttp_outputs>:/orchestrator/outputs
    -v <directory leaf_textures/ttp_data>:/orchestrator/data
    -v <directory leaf_textures/mlflow>:/mlflow
    --name ttp 
    gcr.io/synergos-aisg/synergos_ttp:v0.1.0 
        --id ttp 
        --logging_variant basic
```

**Worker 1**

```
docker run 
    -v <directory leaf_textures/data1>:/worker/data 
    -v <directory leaf_textures/outputs_1>:/worker/outputs 
    --name worker_1 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0 
        --id worker_1 
        --logging_variant basic
```

**Worker 2**

```
docker run 
    -v <directory leaf_textures/data2>:/worker/data 
    -v <directory leaf_textures/outputs_2>:/worker/outputs 
    --name worker_2 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0
        --id worker_2 
        --logging_variant basic
```

#### Synergos UI

- Refer to these [instructions](https://github.com/aimakerspace/synergos_ui) to deploy `Synergos UI`.

Once ready, for each terminal, you should see a REST server running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

### A. Configuring `Synergos Simulator`

All hosts & ports have already been pre-allocated!

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings.

### B. Configuring your manual setup

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- ttp address: `172.17.0.2`
- worker_1 address: `172.17.0.3`
- worker_2 address: `172.17.0.4`
- UI address: `172.17.0.5`

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [1]:
from synergos import Driver

host = "172.18.0.2"
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Registration

Submitting Orchestrator & Participant metadata

#### 1A. Orchestrator creates a collaboration

In [2]:
collab_task = driver.collaborations
collab_task.create('leaf_textures_synbasic_collaboration')

{'data': {'doc_id': '2',
  'kind': 'Collaboration',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration'},
  'relations': {'Project': [],
   'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'collaborations.post',
 'params': {}}

#### 1B. Orchestrator creates a project

In [3]:
driver.projects.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
    }
)

{'data': {'doc_id': '2',
  'kind': 'Project',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration'}}

#### 1C. Orchestrator creates an experiment

In [4]:
driver.experiments.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    model=[
        {
            "activation": "sigmoid",
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 64,
                "out_features": 32
            }
        },
        {
            "activation": "softmax",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 32,
                "out_features": 16
            }
        }
    ]
)

{'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project'},
 'data': {'created_at': '2021-09-14 09:15:45 N',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'expt_id': 'leaf_textures_synbasic_experiment',
   'project_id': 'leaf_textures_synbasic_project'},
  'model': [{'activation': 'sigmoid',
    'is_input': True,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 64, 'out_features': 32}},
   {'activation': 'softmax',
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 32, 'out_features': 16}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 2,
  'kind': 'Experiment'}}

#### 1D. Orchestrator creates a run

In [5]:
driver.runs.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    run_id="leaf_textures_synbasic_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '4',
  'kind': 'Run',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project',
   'expt_id': 'leaf_textures_synbasic_experiment',
   'run_id': 'leaf_textures_synbasic_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project',
  'expt_id': 'leaf_textures_synbasic_experiment'}}

#### 1E. Participants registers their servers' configurations and roles

In [6]:
participant_resp_1 = driver.participants.create(
    participant_id="worker_1",
)

display(participant_resp_1)

participant_resp_2 = driver.participants.create(
    participant_id="worker_2",
)

display(participant_resp_2)

{'data': {'doc_id': '1',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_1'},
  'relations': {'Registration': [{'doc_id': '1',
     'kind': 'Registration',
     'key': {'collab_id': 'imagenette_syncluster_collaboration',
      'project_id': 'imagenette_syncluster_project',
      'participant_id': 'worker_1'},
     'collaboration': {'catalogue': {},
      'logs': {},
      'meter': {},
      'mlops': {},
      'mq': {}},
     'project': {'action': None, 'incentives': {}, 'start_at': None},
     'participant': {'id': None,
      'category': [],
      'summary': None,
      'phone': None,
      'email': None,
      'socials': {}},
     'role': 'host',
     'n_count': 2,
     'node_1': {'host': '172.20.0.7',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000},
     'node_0': {'host': '172.20.0.4',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000}}],
   'Tag': [{'doc_id': '1',
     'kind': 'Tag',
     '

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'worker_2'},
  'relations': {'Registration': [{'doc_id': '2',
     'kind': 'Registration',
     'key': {'collab_id': 'imagenette_syncluster_collaboration',
      'project_id': 'imagenette_syncluster_project',
      'participant_id': 'worker_2'},
     'collaboration': {'catalogue': {},
      'logs': {},
      'meter': {},
      'mlops': {},
      'mq': {}},
     'project': {'action': None, 'incentives': {}, 'start_at': None},
     'participant': {'id': None,
      'category': [],
      'summary': None,
      'phone': None,
      'email': None,
      'socials': {}},
     'role': 'guest',
     'n_count': 2,
     'node_1': {'host': '172.20.0.8',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000},
     'node_0': {'host': '172.20.0.5',
      'port': 8020,
      'log_msgs': True,
      'verbose': True,
      'f_port': 5000}}],
   'Tag': [{'doc_id': '2',
     'kind': 'Tag',
     

In [7]:
registration_task = driver.registrations

# Add and register worker_1 node
registration_task.add_node(
    host='172.18.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    participant_id="worker_1",
    role="host"
)

{'data': {'doc_id': '3',
  'kind': 'Registration',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project',
   'participant_id': 'worker_1'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {},
   'meter': {},
   'mlops': {},
   'mq': {}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_1',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'host',
  'n_count': 1,
  'node_0': {'host': '172.18.0.3',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project',
  'participant_id': 'worker_1'}}

In [8]:
registration_task = driver.registrations

# Add and register worker_2 node
registration_task.add_node(
    host='172.18.0.4',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

registration_task.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    participant_id="worker_2",
    role="guest"
)

{'data': {'doc_id': '4',
  'kind': 'Registration',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project',
   'participant_id': 'worker_2'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue': {},
   'logs': {},
   'meter': {},
   'mlops': {},
   'mq': {}},
  'project': {'action': 'classify',
   'incentives': {'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'worker_2',
   'category': [],
   'summary': None,
   'phone': None,
   'email': None,
   'socials': {}},
  'role': 'guest',
  'n_count': 1,
  'node_0': {'host': '172.18.0.4',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project',
  'participant_id': 'worker_2'}}

#### 1F. Participants registers their tags for a specific project

In [9]:
# Worker 1 declares their data tags
driver.tags.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    participant_id="worker_1",
    train=[["leaf_textures", "dataset", "data1", "train"]],
    evaluate=[["leaf_textures", "dataset", "data1", "evaluate"]]
)

{'data': {'doc_id': '3',
  'kind': 'Tag',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project',
   'participant_id': 'worker_1'},
  'train': [['leaf_textures', 'dataset', 'data1', 'train']],
  'evaluate': [['leaf_textures', 'dataset', 'data1', 'evaluate']],
  'predict': []},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project',
  'participant_id': 'worker_1'}}

In [10]:
# Worker 2 declares their data tags
driver.tags.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    participant_id="worker_2",
    train=[["leaf_textures", "dataset", "data2", "train"]],
    evaluate=[["leaf_textures", "dataset", "data2", "evaluate"]]
)

{'data': {'doc_id': '4',
  'kind': 'Tag',
  'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
   'project_id': 'leaf_textures_synbasic_project',
   'participant_id': 'worker_2'},
  'train': [['leaf_textures', 'dataset', 'data2', 'train']],
  'evaluate': [['leaf_textures', 'dataset', 'data2', 'evaluate']],
  'predict': []},
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project',
  'participant_id': 'worker_2'}}

## Phase 2: 
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [11]:
driver.alignments.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    verbose=False,
    log_msg=False
)

{'data': [{'doc_id': '3',
   'kind': 'Alignment',
   'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
    'project_id': 'leaf_textures_synbasic_project',
    'participant_id': 'worker_2'},
   'train': {'X': [], 'y': []},
   'evaluate': {'X': [], 'y': [6]},
   'predict': {'X': [], 'y': []}},
  {'doc_id': '4',
   'kind': 'Alignment',
   'key': {'collab_id': 'leaf_textures_synbasic_collaboration',
    'project_id': 'leaf_textures_synbasic_project',
    'participant_id': 'worker_1'},
   'train': {'X': [], 'y': [51]},
   'evaluate': {'X': [], 'y': [6]},
   'predict': {'X': [], 'y': []}}],
 'apiVersion': '0.1.0',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'collab_id': 'leaf_textures_synbasic_collaboration',
  'project_id': 'leaf_textures_synbasic_project'}}

#### 2B. Trigger training across the federated grid

In [12]:
model_resp = driver.models.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    run_id="leaf_textures_synbasic_run",
    log_msg=False,
    verbose=False
)
display(model_resp)

{'data': [{'doc_id': '4',
   'kind': 'Model',
   'key': {'project_id': 'leaf_textures_synbasic_project',
    'expt_id': 'leaf_textures_synbasic_experiment',
    'run_id': 'leaf_textures_synbasic_run'},
   'global': {'origin': 'ttp',
    'path': '/orchestrator/outputs/leaf_textures_synbasic_collaboration/leaf_textures_synbasic_project/leaf_textures_synbasic_experiment/leaf_textures_synbasic_run/global_model.pt',
    'loss_history': '/orchestrator/outputs/leaf_textures_synbasic_collaboration/leaf_textures_synbasic_project/leaf_textures_synbasic_experiment/leaf_textures_synbasic_run/global_loss_history.json'},
   'local_2': {'origin': 'worker_2',
    'path': '/orchestrator/outputs/leaf_textures_synbasic_collaboration/leaf_textures_synbasic_project/leaf_textures_synbasic_experiment/leaf_textures_synbasic_run/local_model_worker_2.pt',
    'loss_history': '/orchestrator/outputs/leaf_textures_synbasic_collaboration/leaf_textures_synbasic_project/leaf_textures_synbasic_experiment/leaf_textures

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [13]:
# Orchestrator performs post-mortem validation
driver.validations.create(
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    run_id="leaf_textures_synbasic_run",
    log_msg=False,
    verbose=False
)

{'data': [{'doc_id': '7',
   'kind': 'Validation',
   'key': {'participant_id': 'worker_1',
    'collab_id': 'leaf_textures_synbasic_collaboration',
    'project_id': 'leaf_textures_synbasic_project',
    'expt_id': 'leaf_textures_synbasic_experiment',
    'run_id': 'leaf_textures_synbasic_run'},
   'evaluate': {'statistics': {'accuracy': [0.996875,
      0.990625,
      0.98125,
      0.990625,
      0.9875,
      0.98125,
      1.0,
      0.9875,
      0.996875,
      0.9875,
      0.990625,
      0.984375,
      0.99375,
      0.99375,
      0.25,
      0.99375,
      0.990625,
      0.9875,
      0.9875,
      0.9875,
      0.9875,
      0.990625,
      0.99375,
      0.978125,
      0.99375,
      0.990625,
      0.984375,
      0.9875,
      0.984375,
      0.984375,
      0.99375,
      0.990625,
      0.99375,
      0.9875,
      0.996875,
      0.9875,
      0.990625,
      0.99375,
      0.990625,
      0.99375,
      0.984375,
      0.9875,
      0.98125,
      0.990625,
   

#### 3B. Perform prediction(s) of combination(s)

In [14]:
# Worker 1 requests for inferences
driver.predictions.create(
    tags={"leaf_textures_synbasic_project": [["leaf_textures", "dataset", "data1", "predict"]]},
    participant_id="worker_1",
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    run_id="leaf_textures_synbasic_run"
)

{'data': [{'doc_id': '3',
   'kind': 'Prediction',
   'key': {'participant_id': 'worker_1',
    'project_id': 'leaf_textures_synbasic_project',
    'expt_id': 'leaf_textures_synbasic_experiment',
    'run_id': 'leaf_textures_synbasic_run'},
   'predict': {'statistics': {'accuracy': [0.984375,
      0.9875,
      0.9875,
      0.9875,
      0.996875,
      0.990625,
      0.984375,
      0.990625,
      0.990625,
      0.996875,
      0.99375,
      0.99375,
      1.0,
      0.9875,
      0.209375,
      0.990625,
      0.971875,
      0.99375,
      0.99375,
      0.984375,
      0.9875,
      0.9875,
      0.99375,
      0.99375,
      0.99375,
      0.9875,
      0.996875,
      0.990625,
      0.99375,
      0.99375,
      0.99375,
      0.9875,
      0.98125,
      0.99375,
      0.9875,
      0.990625,
      0.99375,
      0.990625,
      0.9875,
      0.984375,
      1.0,
      0.990625,
      0.99375,
      0.990625,
      0.990625,
      0.996875,
      0.984375,
      0.990625

In [15]:
# Worker 2 requests for inferences
driver.predictions.create(
    tags={"leaf_textures_synbasic_project": [["leaf_textures", "dataset", "data2", "predict"]]},
    participant_id="worker_2",
    collab_id="leaf_textures_synbasic_collaboration",
    project_id="leaf_textures_synbasic_project",
    expt_id="leaf_textures_synbasic_experiment",
    run_id="leaf_textures_synbasic_run",
)

{'data': [{'doc_id': '4',
   'kind': 'Prediction',
   'key': {'participant_id': 'worker_2',
    'project_id': 'leaf_textures_synbasic_project',
    'expt_id': 'leaf_textures_synbasic_experiment',
    'run_id': 'leaf_textures_synbasic_run'},
   'predict': {'statistics': {'accuracy': [0.984375,
      0.9875,
      0.9875,
      0.9875,
      0.996875,
      0.990625,
      0.984375,
      0.990625,
      0.990625,
      0.996875,
      0.99375,
      0.99375,
      1.0,
      0.9875,
      0.209375,
      0.990625,
      0.971875,
      0.99375,
      0.99375,
      0.984375,
      0.9875,
      0.9875,
      0.99375,
      0.99375,
      0.99375,
      0.9875,
      0.996875,
      0.990625,
      0.99375,
      0.99375,
      0.99375,
      0.9875,
      0.98125,
      0.99375,
      0.9875,
      0.990625,
      0.99375,
      0.990625,
      0.9875,
      0.984375,
      1.0,
      0.990625,
      0.99375,
      0.990625,
      0.990625,
      0.996875,
      0.984375,
      0.990625